#### Workbook to read in data from different sources and prep for processing

In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import os
import glob
import requests
import warnings

## Flares data

In [2]:
projcrs = 4326

In [20]:
def read_excel(sheet_name):
    
    all_files = glob.glob("data/rmidata/*.xlsx")
    temp = []

    for filename in all_files:
        df = pd.read_excel(filename, index_col=None, header=0, sheet_name=sheet_name)
        temp.append(df)

    df = pd.concat(temp, axis=0, ignore_index=True)

    # some data cleanup
    df.rename(columns={'BCM 2019':'BCM_2019', 'Avg. temp':'avg_temp',
                       'BCM 2018':'BCM_2018', 'BCM 2020':'BCM_2020',
                       'BCM 2021':'BCM_2021',
                       'Detection_frequency_2012':'Det_freq12',
                       'Detection_frequency_2013':'Det_freq13',
                       'Detection_frequency_2014':'Det_freq14',
                       'Detection_frequency_2015':'Det_freq15',
                       'Detection_frequency_2016':'Det_freq16',
                       'Detection_frequency_2017':'Det_freq17',
                       'Detection frequency 2018':'Det_freq18',
                       'Detection freq. 2019':'Det_freq19',
                       'Detection frequency 2020':'Det_freq20',
                       'Detection frequency 2021':'Det_freq21',
                       'Clear_obs_2012': 'clr_obs12',
                       'Clear_obs_2013': 'clr_obs13',
                       'Clear_obs_2014': 'clr_obs14',
                       'Clear_obs_2015': 'clr_obs15',
                       'Clear_obs_2016': 'clr_obs16',
                       'Clear_obs_2017': 'clr_obs17',
                       'Clear obs 2018': 'clr_obs18',
                       'Clear obs. 2019': 'clr_obs19',
                       'Clear Obs. 2020': 'clr_obs20',
                       'Clear Obs. 2021': 'clr_obs21'}, inplace=True)
    
    df["flare_category"] = sheet_name  # add new column for overall flare type, to make summary stats easier
    
    return(df)

In [21]:
def convert_geopandas(df):
    '''Convert pandas df to geodataframe, drop orig Lat/Long cols, and set crs to common project crs'''
    
    df = gp.GeoDataFrame(
        df, geometry=gp.points_from_xy(df["Longitude"],
                                       df["Latitude"]), 
        crs = projcrs)

    #df.drop(['Latitude', 'Longitude'], axis=1, inplace=True) 
    return(df)

In [22]:
# define sheet names
all_upstream="flares_upstream"
oil_downstream="flares_oil_downstream"
gas_downstream="flares_gas_downstream"

# create dfs by flare category
flares_upstream = read_excel(all_upstream)
oil_downstream = read_excel(oil_downstream)
gas_downstream = read_excel(gas_downstream)

# combine into single df and convert to geodataframe
all_flares = convert_geopandas(pd.concat([flares_upstream, oil_downstream, gas_downstream]))

In [23]:
all_flares.sample(5)

,Country,ISO Code,Catalog ID,id #,Latitude,Longitude,BCM_2019,avg_temp,Ellipticity,Detection_frequency_2019,...,Det_freq15,Det_freq16,Catalog id,BCM_2018,Det_freq18,clr_obs18,flare_category,Detection freq.,Det_freq19,geometry
50014,United States,NaN,NaN,NaN,41.534216,-76.710938,NaN,NaN,1.60150,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (-76.71094 41.53422)
62620,Indonesia,IDN,NaN,11841.0,-3.231368,103.947361,NaN,NaN,1.60150,NaN,...,NaN,NaN,IDN_UPS_2015_103.9474E_3.2314S_v3.0,0.002787,21.250000,80.0,flares_upstream,NaN,NaN,POINT (103.94736 -3.23137)
10065,United States,USA,USA_UPS_2015_102.6018W_47.5891N_v0.2,NaN,47.589108,-102.601770,NaN,NaN,2.77759,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (-102.60177 47.58911)
49890,United States,NaN,NaN,NaN,40.340059,-80.395208,NaN,NaN,1.60150,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (-80.39521 40.34006)
67712,Iran,IRN,NaN,10102.0,36.541405,60.860414,NaN,NaN,1.60150,NaN,...,NaN,NaN,IRN_UPS_2015_60.8604E_36.5414N_v3.0,0.000129,0.821918,365.0,flares_upstream,NaN,NaN,POINT (60.86041 36.54140)


In [24]:
# save as shapefile
all_flares.to_file("data/all_flares.shp")

## Social vulnerability data from CalEnviroScreen

No cleaning necessary; shapefile read in directly to analysis notebooks

## Refineries

No cleaning necessary; csv read in directly to analysis notebooks

## EJScreen data: ACS 5-year Estimates (2017-2021)

In [30]:
# Load census block group level data from EJscreen
# Source: https://www.epa.gov/ejscreen/download-ejscreen-data
# This is the 2017-2021 5-year ACS average data
ejscreen = pd.read_excel("data/CA_EJSCREEN_2022_Full_with_AS_CNMI_GU_VI.xlsx", index_col=None, header=0)

ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_bg = gp.read_file("data/tl_2022_06_bg.shp")  # CA block groups

# read in cleaned and combined flares data
all_flares = gp.read_file("data/all_flares.shp")

In [31]:
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_bg = ca_bg.to_crs(meters_crs)
all_flares = all_flares.to_crs(meters_crs)

In [27]:
# Merge EJscreen data with CA block groups shapefile
ca_bg.rename(columns={'GEOID':'ID'}, inplace=True)  # old:new. match column names for merging
ca_bg['ID'] = ca_bg['ID'].astype(np.int64)

print(f"{len(ejscreen['ID'])} block groups in the EJScreen data \n")
print(f"{len(ca_bg['ID'])} block groups in the CA block group shapefile\n")
ca_bg_joined = pd.merge(ca_bg, ejscreen, on='ID')
print(f"{len(ca_bg['ID'].unique())-len(ca_bg_joined)} block groups are missing after merge")
warnings.filterwarnings('ignore')

# create column of correct area
ca_bg_joined.loc[:, 'shape_area_new'] = ca_bg_joined.geometry.area
ca_bg_joined.rename(columns={'ID':'bg_id'}, inplace=True)  # old:new

25607 block groups in the EJScreen data 

25607 block groups in the CA block group shapefile

0 block groups are missing after merge


In [29]:
# subset flares to only those in Cali
ca_flares = gp.sjoin(all_flares, ca_counties, how = "inner", predicate = 'within')
print(f'Flares found: {len(ca_flares)}')
ca_flares.drop('index_right', axis=1, inplace=True)

Flares found: 117


In [36]:
print(all_flares['flare_cate'].unique())

# reminder: once you subset to CA there are no downstream gas flares
print(ca_flares['flare_cate'].unique())

['flares_upstream' 'flares_oil_downstream' 'flares_gas_downstream']
['flares_upstream' 'flares_oil_downstream']


In [23]:
ca_flares.to_file('data/ca_flares_clean.shp', driver='ESRI Shapefile')

In [8]:
ca_bg_joined.to_file('data/ca_bg_joined_clean.shp', driver='ESRI Shapefile')

In [ ]:
ca_bg.to_file('data/ca_bg_clean.shp', driver='ESRI Shapefile')

## API: 2010 US Census data

### 2/16/23 note: not using census API right now. Maybe later in project. 

Czolowsky et al used: 
total population, Hispanic, minority, non-Hispanic minority, 5y and younger, under 18y, and 75y and older.

Var list

P001001: total population

P004001: total hispanic or latino origin

H008002: Total races tallied for householders — White alone or in combination with one or more other races


In [ ]:
keys = pd.read_csv("api_keys.csv")
census_api_key = keys.loc[keys['api']=='census']['key'].iloc[0]

In [144]:
# helpful vid: https://www.youtube.com/watch?v=LW-M_UC0VTE
# dec/sf1 variables: https://api.census.gov/data/2010/dec/sf1/variables.html
# dec/sf1 example api calls: https://api.census.gov/data/2010/dec/sf1/examples.html
# CA FIPS codes: https://www.weather.gov/hnx/cafips
# FYI California has 8,057 census tracts, 23,212 block groups, and 710,145 blocks

# set search query and parameters
url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,TRACT,P001001,P004001,H008002&for=block:*&in=state:06&in=county:001&in=tract:*&key={0}".format(census_api_key)

# make call to api
response = requests.request("GET", url)

In [145]:
# clean up column names that come from the api
col_names = ["name", "tract_id", "total_pop", "total_hisp", "race_white", "state", "county", "tract", "block"]

# construct df, beg. at first row so headers are skipped
df = pd.DataFrame(columns=col_names, data=response.json()[1:]) 

In [146]:
# check a few rows
df.sample(5)

,name,tract_id,total_pop,total_hisp,race_white,state,county,tract,block
2194,"Block 1007, Block Group 1, Census Tract 4043, ...",404300,0,0,0,06,001,404300,1007
18306,"Block 1014, Block Group 1, Census Tract 4431.0...",443103,12,12,1,06,001,443103,1014
21099,"Block 1065, Block Group 1, Census Tract 4507.4...",450743,0,0,0,06,001,450743,1065
5062,"Block 1005, Block Group 1, Census Tract 4089, ...",408900,0,0,0,06,001,408900,1005
14094,"Block 1066, Block Group 1, Census Tract 4371.0...",437101,0,0,0,06,001,437101,1066
20825,"Block 4028, Block Group 4, Census Tract 4507.0...",450701,0,0,0,06,001,450701,4028
20730,"Block 3168, Block Group 3, Census Tract 4507.0...",450701,0,0,0,06,001,450701,3168
16546,"Block 1267, Block Group 1, Census Tract 4415.0...",441503,0,0,0,06,001,441503,1267
15957,"Block 2015, Block Group 2, Census Tract 4412, ...",441200,48,48,16,06,001,441200,2015
8294,"Block 1005, Block Group 1, Census Tract 4251.0...",425102,0,0,0,06,001,425102,1005


In [147]:
# save to a csv for further processing in other notebooks
df.to_csv("data/ca_blocklevel_demographics.csv")